# **Requisitos**

Antes de executar este notebook, verifique os seguintes pontos:

### **1. Arquivos RAW gerados**
Certifique-se de que, ao rodar o arquivo **ETL.ipynb**, foram gerados corretamente os seguintes arquivos:

- `DENGBR20.csv`  
- `DENGBR21.csv`  
- `DENGBR22.csv`

Esses arquivos serão usados na etapa de **extração** dos dados.

---

### **2. Banco de dados PostgreSQL configurado**
É necessário ter o **PostgreSQL** instalado e uma **database** criada especificamente para este projeto.

Você deve saber e guardar as seguintes informações para configurar a conexão:

- **Usuário**  
- **Senha**  
- **Host** (em geral: `localhost`)  
- **Porta** (padrão: `5432`)  
- **Nome do banco**  

Essas credenciais serão usadas para estabelecer a comunicação entre o notebook e o banco de dados via SQLAlchemy.


## **Instale as dependências**

Usaremos pandas para ler os arquivos brutos .csv e sqlalchemy para criar a conexão com o banco de dados postgreSQL.

In [1]:
pip install pandas sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


## **Configuração da Conexão e Definição dos Arquivos RAW**

Nesta etapa, realizamos duas ações fundamentais para iniciar o processo de ingestão dos dados:

### **1. Configuração da conexão com o PostgreSQL**
Aqui definimos:
- Usuário, senha, host, porta e nome do banco  
- A `DATABASE_URL`, que monta a string de conexão  
- O `engine` do SQLAlchemy, responsável por estabelecer a comunicação entre Python e PostgreSQL  

Com isso, garantimos que o notebook consiga enviar dados para o banco.

---

### **2. Mapeamento dos arquivos RAW**
Criamos um dicionário chamado `RAW_DATA_FILES`, onde cada chave representa o nome da tabela bruta no banco, e cada valor corresponde ao arquivo CSV bruto que será lido.


In [2]:
import pandas as pd
from sqlalchemy import create_engine


#1. Configuração da conexão com o PostgreSQL

DB_USER = 'postgres'
DB_PASSWORD = 'bdpostpost'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'dengue_db'

DATABASE_URL = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(DATABASE_URL)

print("Conexão com o banco de dados estabelecida com sucesso.")


# 2. Arquivos RAW

RAW_DATA_FILES = {
    'dengue_raw_2020': 'DENGBR20.csv',
    'dengue_raw_2021': 'DENGBR21.csv',
    'dengue_raw_2022': 'DENGBR22.csv'
}



Conexão com o banco de dados estabelecida com sucesso.



### **ETAPA DE EXTRACT E LOAD (EL)** ### 

Este bloco executa a ingestão dos dados RAW seguindo duas etapas principais:

### **1. Extract (Extração)** ###
   - Lê cada arquivo CSV bruto listado no dicionário RAW_DATA_FILES.
   - A leitura é feita sem transformações, apenas carregando o conteúdo original.
   - Linhas problemáticas são ignoradas com `on_bad_lines='skip'`.

### **2. Load (Carga)** ###
   - Cada DataFrame é inserido diretamente em uma tabela no PostgreSQL.
   - O nome da tabela é definido pela chave no dicionário RAW_DATA_FILES.
   - O parâmetro `if_exists='replace'` recria a tabela a cada execução.
   - Usa-se o método `to_sql()` do pandas, conectando via SQLAlchemy.

In [3]:
print("Iniciando processo de ingestão dos dados RAW...\n")

#  extract e load (EL)
for table_name, file_path in RAW_DATA_FILES.items():
    try:
        print(f"Lendo arquivo: {file_path}")

        # Leitura do CSV sem transformações
        df_raw = pd.read_csv(
            file_path,
            encoding='utf-8',
            on_bad_lines='skip'
        )

        print(f"Inserindo {len(df_raw)} registros na tabela '{table_name}'...")

        # Load direto para o banco
        df_raw.to_sql(
            table_name,
            con=engine,
            if_exists='replace',  # substitui sempre a tabela
            index=False
        )

        print(f"Tabela '{table_name}' carregada com sucesso.\n")

    except Exception as error:
        print(f"ERRO ao processar '{file_path}': {error}\n")

print("Ingestão dos dados RAW finalizada com sucesso.")


Iniciando processo de ingestão dos dados RAW...

Lendo arquivo: DENGBR20.csv


C:\Users\edvyn\AppData\Local\Temp\ipykernel_17800\2216145505.py:9: DtypeWarning: Columns (22,44,45,46,54,56,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(


Inserindo 1495117 registros na tabela 'dengue_raw_2020'...
Tabela 'dengue_raw_2020' carregada com sucesso.

Lendo arquivo: DENGBR21.csv


C:\Users\edvyn\AppData\Local\Temp\ipykernel_17800\2216145505.py:9: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(


Inserindo 1010359 registros na tabela 'dengue_raw_2021'...
Tabela 'dengue_raw_2021' carregada com sucesso.

Lendo arquivo: DENGBR22.csv


C:\Users\edvyn\AppData\Local\Temp\ipykernel_17800\2216145505.py:9: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(


Inserindo 1393877 registros na tabela 'dengue_raw_2022'...
Tabela 'dengue_raw_2022' carregada com sucesso.

Ingestão dos dados RAW finalizada com sucesso.
